In [1]:
import optuna
from pysr import PySRRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [4]:
# 1. Prepare toy data
X = np.random.randn(200, 5)
y = 2.5 * np.cos(X[:, 2]) + X[:, 0]**2 - 0.3 * X[:, 1]

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2. Define Optuna objective
def objective(trial: optuna.Trial) -> float:
    # Suggest PySR hyperparameters
    pop_size = trial.suggest_int("population_size", 50, 500)
    n_iters = trial.suggest_int("niterations", 10, 100)
    parsimony = trial.suggest_float("parsimony", 0.001, 0.1, log=True)
    optimizer_restarts = trial.suggest_int("optimizer_nrestarts", 1, 10)

    model = PySRRegressor(
        population_size=pop_size,
        niterations=n_iters,
        parsimony=parsimony,
        optimizer_nrestarts=optimizer_restarts,
        # you can add more PySR settings here...
    )

    # Fit and evaluate
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    return mean_squared_error(y_valid, y_pred)

# 3. Create and run the study
sampler = optuna.samplers.TPESampler()  # default TPE sampler :contentReference[oaicite:2]{index=2}
pruner = optuna.pruners.MedianPruner()  # stop unpromising trials early :contentReference[oaicite:3]{index=3}

study = optuna.create_study(
    storage = "sqlite:///db.sqlite3",
    study_name = "pysr-test_study",
    direction="minimize",
    sampler=sampler,
    pruner=pruner
)
study.optimize(objective, n_trials=50, timeout=600)

print("Best MSE:", study.best_value)
print("Best params:", study.best_params)

[I 2025-04-27 11:03:37,426] A new study created in RDB with name: pysr-test_study
e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.610e+05
Progress: 63 / 341 total iterations (18.475%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.521e+00  1.594e+01  y = 2.5392
5           1.246e+00  1.761e-01  y = (x₀ * x₀) + 1.5219
7           1.178e+00  2.828e-02  y = (x₀ * (x₀ / 1.2258)) + 1.7094
9           1.154e+00  1.024e-02  y = (x₁ * -0.3257) + ((x₀ * x₀) + 1.5293)
11          1.089e+00  2.902e-02  y = ((x₁ * -0.31956) + ((x₀ * 0.82043) * x₀)) - -1.7118
15          5.849e-01  1.553e-01  y = (-0.32566 * (x₂ * x₂)) + ((x₀ * (0.88948 * x₀)) - (-1....
                                      5293 / 0.90682))
───────────────────────────────────────────────────────────────────────────────────────────────────
══════════════════════════════════════════════════════════════════════════════

[ Info: Final population:
[ Info: Results saved to:
[I 2025-04-27 11:04:03,597] Trial 0 finished with value: 0.020423910166040882 and parameters: {'population_size': 283, 'niterations': 11, 'parsimony': 0.03237795731318641, 'optimizer_nrestarts': 10}. Best is trial 0 with value: 0.020423910166040882.
e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!


  - outputs\20250427_110339_RLwBcB\hall_of_fame.csv

Expressions evaluated per second: 2.480e+05
Progress: 94 / 2821 total iterations (3.332%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.521e+00  1.594e+01  y = 2.5392
5           1.246e+00  1.761e-01  y = (x₀ * x₀) + 1.5221
7           1.178e+00  2.828e-02  y = ((x₀ * 0.81575) * x₀) + 1.7095
9           1.154e+00  1.024e-02  y = ((x₀ * x₀) - -1.5293) - (x₁ * 0.32563)
11          1.263e-01  1.106e+00  y = (x₀ * x₀) + ((x₂ * (x₂ * -0.83902)) + 2.3284)
13          1.235e-01  1.119e-02  y = (x₀ * (x₀ + 0.052602)) + ((x₂ * (x₂ * -0.83625)) + 2.3...
                                      286)
15          1.235e-01  5.960e-08  y = ((x₀ * (x₀ + 0.052607)) - 0.19482) + ((x₂ * (x₂ * -0.8...
                                      3625)) +

[ Info: Final population:
[ Info: Results saved to:


...


Expressions evaluated per second: 1.310e+05
Progress: 60 / 1736 total iterations (3.456%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.521e+00  1.594e+01  y = 2.5392
5           1.246e+00  1.761e-01  y = (x₀ * x₀) + 1.5221
7           1.178e+00  2.828e-02  y = (x₀ * (x₀ / 1.2259)) - -1.7095
9           1.154e+00  1.024e-02  y = ((x₁ * -0.32565) + 1.5293) + (x₀ * x₀)
11          1.263e-01  1.106e+00  y = 2.3284 - (((x₂ * 0.83902) * x₂) - (x₀ * x₀))
13          1.235e-01  1.119e-02  y = 2.3286 - (((x₂ / 1.1958) * x₂) - ((x₀ + 0.052629) * x₀...
                                      ))
17          5.017e-02  2.252e-01  y = ((((x₂ * x₂) * 0.84046) - (x₀ * x₀)) * -0.98471) - ((x...
                                      ₁ * 0.29372) + -2.3403)
───────────────────────────────────────

[ Info: Final population:
[ Info: Results saved to:
[I 2025-04-27 11:13:06,460] Trial 5 finished with value: 0.0001543608617401771 and parameters: {'population_size': 274, 'niterations': 56, 'parsimony': 0.0013126993641312664, 'optimizer_nrestarts': 9}. Best is trial 4 with value: 1.142247087914398e-07.
e:\Documents\Coding\Repositorios\MyRepositories\NewMortalityCurves\.venv\Lib\site-packages\pysr\sr.py:2774: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 1.460e+05
Progress: 88 / 2914 total iterations (3.020%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.521e+00  1.594e+01  y = 2.5392
5           1.246e+00  1.761e-01  y = (x₀ * x₀) + 1.5221
7           1.178e+00  2.828e-02  y = (x₀ * (x₀ * 0.81575)) + 1.7095
9           1.675e-01  9.751e-01  y = (x₀ * x₀) + (2.4832 - (x₂ * x₂))
11          1.263e-01  1.413e-01  y = ((x₀ * x₀) - (x₂ * (x₂ / 1.1919))) + 2.3284
13          1.235e-01  1.119e-02  y = ((x₀ * (x₀ + 0.052603)) - (x₂ * (x₂ / 1.1958))) - -2.3...
                                      286
15          1.235e-01  -0.000e+00  y = (x₀ * (x₀ + 0.052569)) + (((0.24397 - (x₂ * x₂)) / 1....
                                       1958) - -2.1245)
17          5.064e-02  4.456e-01  y = (((x₀ * x₀) +

[ Info: Final population:
[ Info: Results saved to:
[I 2025-04-27 11:15:03,106] Trial 6 finished with value: 0.00015436025648980035 and parameters: {'population_size': 237, 'niterations': 94, 'parsimony': 0.0027556508691657182, 'optimizer_nrestarts': 4}. Best is trial 4 with value: 1.142247087914398e-07.


Best MSE: 1.142247087914398e-07
Best params: {'population_size': 386, 'niterations': 98, 'parsimony': 0.004604403381777251, 'optimizer_nrestarts': 9}


In [3]:
study.best_params

{'population_size': 407,
 'niterations': 77,
 'parsimony': 0.08919810761494844,
 'optimizer_nrestarts': 6}